In [33]:
import pandas as pd

hom_path = "data\\home-credit-default-risk\\HomeCredit_columns_description.csv"  # 列描述表
hom = pd.read_csv(hom_path)
def missing_values_table(df, table_name):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(
        columns={0: 'Missing Values',
                 1: '% of Total Values'})
    miss_num = (mis_val_table["Missing Values"] != 0).sum()
    print("Total " + str(miss_num) + " columns missing values")
    mis_val_table = mis_val_table.merge(
        hom, how="left", left_index=True, right_on='Row')
    mis_val_table = mis_val_table.drop(columns=['Unnamed: 0'])
    mis_val_table = mis_val_table.drop(
        index=mis_val_table.loc[mis_val_table["Table"] != table_name].index)
    mis_val_table = mis_val_table.reindex(
        columns=["Row", "Description", "Special", "Missing Values", "% of Total Values"])
    mis_val_table = mis_val_table.reset_index(drop=True)
    mis_val_table = mis_val_table.merge(
        df.describe().T, how="left", left_on="Row", right_index=True)
    return mis_val_table

source_dir="data\\home-credit-default-risk"
app_tr_path = source_dir+"\\application_train.csv"
app_tr = pd.read_csv(app_tr_path)
app_tr


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
t = missing_values_table(app_tr, "application_{train|test}.csv")
t.reset_index(inplace=True)
t.set_index("Row", inplace=True)

t = pd.concat([app_tr.iloc[0, :], app_tr.dtypes, t], axis=1)
pd.set_option("display.max_rows", 400)
pd.set_option('max_colwidth', 400)
t.index.name = 'Row'
t.drop(index=t.loc[t["Special"]=="normalized"].index,inplace=True) ##得不到归一化的值
t.drop(index=t.loc[t['Description'].str.contains("Number of enquiries to Credit Bureau")].index,inplace=True) #得不到用户的该值
t.drop(index=t.loc[t['Description'].str.contains("Did client provide document")].index,inplace=True) #得不到用户的该值
t.drop(index=t.loc[t['Description'].str.contains("How many observation of client's social surroundings")].index,inplace=True) #得不到用户的该值
t.drop(index=t.loc[t.index=="SK_ID_CURR"].index,inplace=True) #无意义
t.drop(index=t.loc[t.index=="REGION_POPULATION_RELATIVE"].index,inplace=True) #无意义
t.drop(index=t.loc[t.index=="TARGET"].index,inplace=True) #无意义
cols = t.columns.values
cols[0] = 'Example'
cols[1] = 'Type'
cols[2] = 'Index'
t.columns = cols
t = t.reindex(columns=["Example", "Type","Index","Description"])

t.to_csv("ui_item_res.csv")
t


Total 67 columns missing values


,Example,Type,Index,Description
Row,,,,
NAME_CONTRACT_TYPE,Cash loans,object,2,Identification if loan is cash or revolving
CODE_GENDER,M,object,3,Gender of the client
FLAG_OWN_CAR,N,object,4,Flag if the client owns a car
FLAG_OWN_REALTY,Y,object,5,Flag if client owns a house or flat
CNT_CHILDREN,0,int64,6,Number of children the client has
AMT_INCOME_TOTAL,202500.0,float64,7,Income of the client
AMT_CREDIT,406597.5,float64,8,Credit amount of the loan
AMT_ANNUITY,24700.5,float64,9,Loan annuity
AMT_GOODS_PRICE,351000.0,float64,10,For consumer loans it is the price of the goods for which the loan is given
